In [ ]:
#########               taken from https://github.com/ravikiran-mane/FBCNet

In [1]:
"""Get Data from Board"""


In [4]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd


"""Create the CNN Model"""
class CNN(nn.Module):  
    def __init__(self, input_size, kernel_size, conv_channels, 
                 dense_size, dropout):         
        super(CNN, self).__init__()                  
        self.cconv   = []  
        self.MaxPool = nn.MaxPool2d((1, 2), (1, 2))  
        self.ReLU    = nn.ReLU()
        self.Dropout = nn.Dropout(dropout)        
        self.batchnorm = []                
        # ############ batchnorm ###########
        for i in conv_channels:
            self.batchnorm.append(nn.BatchNorm2d(i, eps=0.001, momentum=0.01,
                                                 affine=True, track_running_stats=True).cuda())     
        j = 0 ##### define CONV layer architecture: #####
        for in_channels, out_channels in zip(conv_channels, conv_channels[1:]):                           
            conv_i = nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                                     kernel_size = kernel_size[j], #stride = (1, 2),
                                     padding     = (kernel_size[j][0]//2, kernel_size[j][1]//2))            
            self.cconv.append(conv_i)                
            self.add_module('CNN_K{}_O{}'.format(kernel_size[j], out_channels), conv_i)
            j += 1                            
        self.flat_dim = self.get_output_dim(input_size, self.cconv)    
        self.fc1 = nn.Linear(self.flat_dim, dense_size)
        self.fc2 = nn.Linear(dense_size, 2)                

    def get_output_dim(self, input_size, cconv):        
        with torch.no_grad():
            input = torch.ones(1,*input_size)              
            for conv_i in cconv:                
                input = self.MaxPool(conv_i(input))        
                flatout = int(np.prod(input.size()[1:]))
                print("Input shape : {} and flattened : {}".format(input.shape, flatout))
        return flatout 
        
    def forward(self, input):        
        for i, conv_i in enumerate(self.cconv):
            input = conv_i(input)
            input = self.batchnorm[i+1](input)
            input = self.ReLU(input)        
            input = self.MaxPool(input)                   
        # flatten the CNN output     
        out = input.view(-1, self.flat_dim) 
        out = self.fc1(out)                       
        out = self.Dropout(out)        
        out = self.fc2(out)      
        return out  

In [5]:
"""Tune Hyperparameters"""

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

In [6]:
num_epochs = 150 
learning_rate = 1e-3
weight_decay = 1e-4  
batch_size = 64
verbose = 1

# define kernel size in terms of ms length 
fs = 160 #Hz
time_window = 100 #ms
width = time_window*fs//1000 

# convolution parameters 
h, w = 3, 1

kernel_size = [(h, w*width), (h, w*width), (h, w*width),(h, w*width), (h, w*width), (h, w*width)]
conv_chan = [1, 8, 16, 32, 64] 

table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 

In [ ]:
# Define the architecture

#input_size = DEFINE AN INPUT SIZE FOR DATA

model = CNN(input_size    = input_size, 
              kernel_size   = kernel_size, 
              conv_channels = conv_chan,
              dense_size    = 256, 
              dropout       = 0.5)  

In [7]:
# optimizer and the loss function definition 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
criterion = torch.nn.CrossEntropyLoss()

model.to(dev)  
criterion.to(dev)    
print("Model architecture >>>", model)


NameError: name 'model' is not defined

In [ ]:
"""Training Loop"""
from Train_Model import train_model
best_model, train_losses, val_losses, train_accs, val_accs, info =\
#    train_model(model, data['dset_loaders'], data['dset_sizes'], 
#                criterion, optimizer, dev, lr_scheduler = None, num_epochs = num_epochs, verbose = verbose)    
